# Carga de datos

In [1]:
CLUSTER_EVAL = 92

In [48]:
import pandas as pd
import numpy as np
import gzip, pickle, pickletools
import time
import cloudpickle as cp
import urllib.request

In [3]:
basePath =  'https://www.dropbox.com/s/'

In [4]:
# Dataframe de préstamos
df_prestamos = pd.read_json(basePath + 'chk4h90xgm48adm/joinTablas.json?dl=1')

In [39]:
df_prestamos

,RowID,Fecha,IDItem,NumeroUbicacion,Dewey,Ubicacion,Llave,Programa,Facultad,IDUsuario,Year,Signatura,TipoItem,Autor,Titulo,AnioPublicacion,DeweyEspecifico,TemaDewey,Temas,Union
0,Row0,1547050497000,80000005327627,720.98 A71S,720,COL-GRAL-2,866245,CARRERA DE ARQUITECTURA,arquitectura y diseño,c361c772cd0220bb16dcfb2ad803e4903334ab94,2019,720,LIBRO,"Arango Cardinal, Silvia 1948-",Ciudad y arquitectura seis generaciones que co...,2012.0,720.98,Arquitectura latinoamericana,"[arquitectur, histori, amer, latin]",arquitectur histori amer latin
1,Row1,1547050500000,80000001191496,720.9861 A71,720,COL-GRAL-2,309945,CARRERA DE ARQUITECTURA,arquitectura y diseño,c361c772cd0220bb16dcfb2ad803e4903334ab94,2019,720,LIBRO,"Arango Cardinal, Silvia 1948-",Historia de la arquitectura en Colombia Silvia...,1993.0,720.9861,Arquitectura colombiana,"[arquitectur, colonial, histori, colombi]",arquitectur colonial histori colombi
2,Row2,1547138763000,80000004979759,540 CH15Q 2010,540,COL-GRAL-3,822727,CARRERA DE ARQUITECTURA,arquitectura y diseño,87b0e5a61ed712ddfaf5d478ad68c87c825997e9,2019,540,LIBRO,"Chang, Raymond",Química Raymond Chang ; revisión técnica Rosa ...,2010.0,540,Química,[quimic],quimic
3,Row3,1547138765000,80000004509512,512 B151,510,COL-GRAL-3,784541,CARRERA DE ARQUITECTURA,arquitectura y diseño,87b0e5a61ed712ddfaf5d478ad68c87c825997e9,2019,512,LIBRO,"Baldor, J. Aurelio",Álgebra con gráficos y 6523 ejercicios y probl...,2009.0,512,Álgebra,[algebr],algebr
4,Row4,1547300269000,80000005270756,712 M17E,710,COL-GRAL-2,858473,CARRERA DE ARQUITECTURA,arquitectura y diseño,afe8c17ba351d274fc5671978918f0c93032d88f,2019,712,LIBRO,"Martínez de Pisón, Eduardo 1937-",Miradas sobre el paisaje Eduardo Martínez de P...,2009.0,712,Arquitectura del paisaje,"[arquitectur, paisaj]",arquitectur paisaj
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489732,Row543896,1546275870000,80000005573398,370.1 E47E,370,COL-GRAL-3,914491,CARRERA DE ARTES VISUALES,artes,7544d4d6cf0b97aa221bcefdb69d3c8be2b4e2d4,2018,370,LIBRO,"Eisner, Elliot W.",La escuela que necesitamos ensayos personales ...,2002.0,370.1,Filosofía de la educación,"[filosofi, educacion]",filosofi educacion
489733,Row543897,1546275870000,60000000046504,370.1 CH65,370,COL-GRAL-3,747839,CARRERA DE ARTES VISUALES,artes,7544d4d6cf0b97aa221bcefdb69d3c8be2b4e2d4,2018,370,LIBRO,"Chomsky, Noam 1928-",La (des) educación Noam Chomsky ; edición e in...,2007.0,370.1,Filosofía de la educación,"[filosofi, educacion]",filosofi educacion
489734,Row543898,1546275870000,80000002767872,809 H182 V.1,800,COL-GRAL-0,522997,CARRERA DE ARTES VISUALES,artes,7544d4d6cf0b97aa221bcefdb69d3c8be2b4e2d4,2018,809,LIBRO,"Hauser, Arnold 1892-1978",Historia social de la literatura y el arte Arn...,1998.0,809,Literatura europea|xHistoria y crítica,"[arte, histori]",arte histori
489735,Row543899,1546284470000,80000004114631,658.0076 G51,650,COL-GRAL-2,743895,CARRERA DE RELACIONES INTERNACIONALES,ciencias políticas y relaciones internacionales,9203547282e98eb313fa78461c3530828f04bb4b,2018,658,LIBRO,None,GMAT premier program the staff of kaplan test ...,2007.0,658.0076,Examen de admisión para graduados en administr...,"[exam, admision, gradu, administracion, empres]",exam admision gradu administracion empres


In [5]:
# Dataframe de datos de material
df_temas = pd.read_json(basePath + 'a2wops7yxdjd2ea/Libros.json?dl=1')

In [15]:
# Dataframe de pesos por los temas de cada cluster6
df_pesos_cluster_temas = pd.read_json(basePath + 'nbz30whibx2r67s/pesos_cluster_temas.json?dl=1')

In [16]:
# Dataframe de pesos por los deweys de cada cluster
df_pesos_cluster_deweys = pd.read_json(basePath + 'f5ixzg087pd8gca/pesos_cluster_deweys.json?dl=1')

# Funciones auxiliares

In [40]:
#Para obtener el nombre de un material a partir del título
def itemIdToMaterial(id):
    return df_prestamos.loc[df_prestamos['Llave'] == id]['Titulo'].unique()

In [41]:
def itemIdToDewey(id):
    item = (df_prestamos.loc[df_prestamos['Llave'] == id]['Dewey'].unique())
    if len(item) == 0:
        return  (df_prestamos.loc[df_prestamos['Llave'] == str(id)]['Dewey'].unique())[0]
    else:
        return item[0]

In [42]:
def itemIdToThemes(id):
    tematicas = df_temas.loc[df_temas['Llaves'] == str(id)]['Tema 670'].values
    if len(tematicas) == 0:
        return []
    else:
        return tematicas

In [43]:
def generateClusterInfo(cluster_eval):
    dfP = df_pesos_cluster_temas.copy()
    user_p = dfP.loc[dfP['cluster'] == cluster_eval]
    tematicas =[]
    for llave in user_p['Llaves'].unique():
        tematicas.append(itemIdToThemes((llave)))

    user_p['Tematicas'] = tematicas
    display(user_p)
    deweys = ((dfP.loc[dfP['cluster'] == cluster_eval]['Dewey'].unique()))
    print((dfP.loc[dfP['cluster'] == cluster_eval]['Dewey'].unique()))

    tematicas = []
    for tems in user_p['Tematicas']:
        for tema in tems:
            for temasub in tema:
                if temasub in tematicas:
                    continue
                else:
                    tematicas.append(temasub)

    return user_p, deweys, set(tematicas)

In [44]:
def generateRecsInfopreds(preds):
    recs = preds
    data_recs_asr = []
    for ids in recs:
        data_recs_asr.append({'Titulo':itemIdToMaterial(ids), 
                         #'Rating':ratings[1], 
                          'Dewey':itemIdToDewey(ids), 
                         'Tematicas':itemIdToThemes(ids),
                          'Llave': ids})

    df_recs_asr = pd.DataFrame(data_recs_asr)
    display(df_recs_asr)
    deweys = print((df_recs_asr['Dewey'].unique()))
    print((df_recs_asr['Dewey'].unique()))

    tematicas = []
    for tems in df_recs_asr['Tematicas']:
        for tema in tems:
            for temasub in tema:
                if temasub in tematicas:
                    continue
                else:
                    tematicas.append(temasub)

    return df_recs_asr, deweys, set(tematicas)

In [45]:
# Compara las tematicas de los ítems recomendados con respecto a los prestamos que ha realizado el cluster
def compare_recs(setRecs, setPrest):
    difference = setRecs - setPrest
    intersection = setRecs & setPrest
    print("Diferencia: ", difference)
    print("Intersección: ", intersection)
    return difference, intersection

In [23]:
df_info_clus, df_clus_deweys, df_clus_tems = generateClusterInfo(CLUSTER_EVAL)

/anaconda/envs/reco_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Llaves,cluster,Dewey,Peso del prestamos,timestamp,Tematicas
1692,255776,92,190,0.250,2018-08-03,[]
1695,255779,92,190,0.250,2019-03-19,"[[filosofía, francés]]"
1701,255783,92,800,0.250,2019-02-13,"[[literatura, moral]]"
1850,256000,92,190,0.125,2019-05-22,"[[filosofía, alemán], [filosofía, alemán]]"
1864,256034,92,130,0.125,2018-11-08,[]
...,...,...,...,...,...,...
179612,885340,92,840,1.000,2019-11-19,"[[poesía, francés]]"
179635,885617,92,840,0.500,2019-01-21,"[[cuento, francés]]"
182378,908913,92,270,0.250,2019-01-21,"[[cristianismo, historia]]"
183156,915537,92,110,0.750,2019-02-06,"[[filosofía, naturaleza]]"


['190' '800' '130' '880' '740' '110' '140' '180' '840' '260' '710' '810'
 '720' '860' '330' '920' '400' '750' '150' '890' '370' '820' '300' '320'
 '780' '170' '520' '590' '20' '500' '120' '830' '0' '460' 'Video' '700'
 '570' '900' '580' '850' '440' '270' '340' '610' '980']


# Modelos

## SAR

In [24]:
sar_model_path = basePath + "5brcaex6fdqs3yc/sar_predictor_temas_100k_10011910.pkl?dl=1"

In [35]:
with gzip.open(urllib.request.urlopen(sar_model_path), 'rb') as f:
    p = pickle.Unpickler(f)
    sar_predictor = p.load()

/anaconda/envs/reco_gpu/lib/python3.7/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [37]:
sar_preds = sar_predictor.predict_for_cluster(cluster=CLUSTER_EVAL)

In [46]:
df_info_pred_sar, df_pred_sar_deweys, df_pred_sar_tems = generateRecsInfopreds(sar_preds)

,Titulo,Dewey,Tematicas,Llave
0,[Entre nosotros ensayos para pensar en otro Em...,190,"[[filosofía, francés]]",341300
1,[La filosofía de Martín Heidegger Alfonso de W...,190,"[[filosofía, alemán]]",271907
2,"[Deleuze, Whitehead, Bergson rhizomatic connec...",190,"[[filosofía, francés]]",795173
3,[Deshoras Julio Cortázar],860,"[[cuento, argentino]]",678233
4,[Rizoma introducción Gilles Deleuze y Pierre F...,190,"[[filosofía, francés]]",341323
5,[Memorias del subsuelo Fédor Mikhailovitch Dos...,890,"[[novela, ruso]]",725775
6,[Compendio de psicología William James],150,[[psicología]],330489
7,"[Obras filosoficas Discurso del metodo, Tratad...",190,"[[filosofía, francés]]",272620
8,[Gilles Deleuze Claire Colebrook],190,"[[filosofía, francés]]",797150
9,"[El multiculturalismo y ""la política del recon...",300,[[multiculturalismo]],347754


['190' '860' '890' '150' '300']
['190' '860' '890' '150' '300']


In [47]:
compare_recs(df_pred_sar_tems, df_clus_tems)

Diferencia:  set()
Intersección:  {'alemán', 'multiculturalismo', 'psicología', 'filosofía', 'novela', 'francés', 'cuento', 'ruso', 'argentino'}


(set(),
 {'alemán',
  'argentino',
  'cuento',
  'filosofía',
  'francés',
  'multiculturalismo',
  'novela',
  'psicología',
  'ruso'})

## Content Based

In [53]:
content_model_path = basePath + "jejvttr9i0xfl2w/content_predictor_temas_60k_10012110.pkl?dl=1"

In [54]:
with gzip.open(urllib.request.urlopen(content_model_path), 'rb') as f:
    p = pickle.Unpickler(f)
    content_predictor = p.load()

In [55]:
content_preds = content_predictor.predict_for_cluster(cluster=CLUSTER_EVAL)

1

Usando el recomendador:  ContentKNN
Computing recommendations...


In [56]:
df_info_content_sar, df_pred_content_deweys, df_pred_content_tems = generateRecsInfopreds(content_preds)

,Titulo,Dewey,Tematicas,Llave
0,[La situación en el mundo un informe del World...,330,[],342352
1,[Experiencia e historia escritos de juventud F...,190,[],350219
2,[Historia de la lengua española Rafael Lapesa],460,"[[español, historia]]",275733
3,[Desmovilizar a los paramilitares en Colombia ...,320,"[[paramilitar, colombia]]",677027
4,[Teoría de juegos una introducción no técnica ...,510,"[[teoría, juegos]]",266159
5,[Manual de urbanidad y buenas maneras de consu...,390,[],318787
6,[Les chants de maldoror Comte de Lautreamont],840,"[[poesía, francés]]",346966
7,[Guerras justas e injustas un razonamiento mor...,350,"[[guerrar, aspecto, moral, ético]]",635877
8,[Clinical nutrición Victoria F. Thiele],610,[],264237
9,[Entre nosotros ensayos para pensar en otro Em...,190,"[[filosofía, francés]]",341300


['330' '190' '460' '320' '510' '390' '840' '350' '610']
['330' '190' '460' '320' '510' '390' '840' '350' '610']


In [57]:
compare_recs(df_pred_content_tems, df_clus_tems)

Diferencia:  {'guerrar', 'paramilitar', 'juegos', 'teoría'}
Intersección:  {'moral', 'aspecto', 'poesía', 'español', 'francés', 'historia', 'colombia', 'ético', 'filosofía'}


({'guerrar', 'juegos', 'paramilitar', 'teoría'},
 {'aspecto',
  'colombia',
  'español',
  'filosofía',
  'francés',
  'historia',
  'moral',
  'poesía',
  'ético'})

## SVD

In [58]:
svd_model_path = basePath + "x6kp4sm4rnu7h7y/svd_predictor100k_10012010.pkl?dl=1"

In [59]:
with gzip.open(urllib.request.urlopen(svd_model_path), 'rb') as f:
    p = pickle.Unpickler(f)
    svd_predictor = p.load()

System version: 3.7.11 (default, Jul 27 2021, 14:32:16) 
[GCC 7.5.0]
Surprise version: 1.1.1


In [60]:
svd_preds = svd_predictor.predict_for_cluster(cluster=CLUSTER_EVAL)

In [61]:
df_info_svd_sar, df_pred_svd_deweys, df_pred_svd_tems = generateRecsInfopreds(svd_preds)

,Titulo,Dewey,Tematicas,Llave
0,[Neuroanatomía clínica Richard S. Snell],610,[[neuroanatomía]],801231
1,[Microbiología de las infecciones humanas edit...,610,"[[microbiología, médico]]",725351
2,[El embrujo de Shanghái Juan Marsé],860,"[[novela, español]]",855109
3,[New critical legal thinking law and the polit...,340,"[[estudio, crítico, derecho]]",861995
4,[Convivir con tu embarazo y con tu bebé Franci...,610,[],833865
5,[La evaluación de la acción y de las políticas...,350,"[[política, público, evaluación]]",718174
6,[Hacia una sociología del discurso pedagogico ...,370,[[sociolingüístico]],534834
7,[Las claves del ensayo Fernando Vásquez Rodríg...,800,"[[ensayo, técnico]]",918213
8,[Reglamento colombiano de construcción sismo r...,690,"[[construcción, antisísmica, norma, técnico], ...",919896
9,[Introducción a la macroeconomía colombiana Co...,330,"[[macroeconomía, colombia]]",345661


['610' '860' '340' '350' '370' '800' '690' '330']
['610' '860' '340' '350' '370' '800' '690' '330']


In [62]:
compare_recs(df_pred_svd_tems, df_clus_tems)

Diferencia:  {'evaluación', 'antisísmica', 'público', 'construcción', 'derecho', 'microbiología', 'médico', 'economía', 'macroeconomía', 'estudio', 'norma', 'neuroanatomía', 'política', 'sociolingüístico'}
Intersección:  {'ensayo', 'español', 'novela', 'crítico', 'colombia', 'técnico', 'ética'}


({'antisísmica',
  'construcción',
  'derecho',
  'economía',
  'estudio',
  'evaluación',
  'macroeconomía',
  'microbiología',
  'médico',
  'neuroanatomía',
  'norma',
  'política',
  'público',
  'sociolingüístico'},
 {'colombia', 'crítico', 'ensayo', 'español', 'novela', 'técnico', 'ética'})